# Experiment 6
## Database connectivity in Python

### a) Write python code to perform CRUD (create, read, update and delete) operations on a database

In [1]:
import sqlalchemy as db

def db_connect(server, user, passwd, dbname):
    """Connect to the DB and return the engine and connection objects"""
    try:
        engine = db.create_engine("mysql+pymysql://{user}:{pwd}@{host}/{dbname}".format(user=user, pwd=passwd, host=server, dbname=dbname))
        conn = engine.connect()
    except:
        print("Couldn't connect to the database")
        return None
    return engine, conn

def get_table(engine, metadata, tablename):
    """Get the specified table object"""
    # Reflect table from the engine
    return db.Table(tablename, metadata, autoload=True, autoload_with=engine)

def display_table_records(engine, table, size = None):
    """Display records of the specified table"""
    stmt = db.select(table)
    print(stmt)
    if size == None:
        results = engine.execute(stmt).fetchall()
    else:
        results = engine.execute(stmt).fetchmany(size=size)
    
    if len(results) == 0:
        print("Table is empty")
    else:
        print(table.columns.keys())
        for record in results:
            print(record)

def truncate_table(engine, table):
    stmt = db.delete(table)
    engine.execute(stmt)

In [2]:

# connect to the DB
engine, conn = db_connect(server = "localhost:3306", user = "root", passwd = "", dbname = "Employees")
print(type(engine))
print(type(conn))

# Create a metadata object
metadata = db.MetaData()

# employees table
employees = get_table(engine, metadata, 'employees')
print(employees.columns.keys())
print(type(employees))
truncate_table(engine, employees)

<class 'sqlalchemy.engine.base.Engine'>
<class 'sqlalchemy.engine.base.Connection'>
['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
<class 'sqlalchemy.sql.schema.Table'>


In [3]:
# insert
display_table_records(engine, employees)

ins = employees.insert().values({'emp_no':1001, 'birth_date':'19851012', 'first_name':'John', 'last_name':'Doe', 'gender':'M', 'hire_date':'20150901'})
print(type(ins))
print(str(ins))

try:
    engine.execute(ins)
except BaseException as e:
    print("Error: ", e)

display_table_records(engine, employees)

rec2 = {'emp_no':1002,
        'birth_date':'20011123',
        'first_name':'Jane',
        'last_name':'Doe',
        'gender':'F',
        'hire_date':'20200301'
       }
ins = employees.insert().values(rec2)
try:
    engine.execute(ins)
except BaseException as e:
    print("Error: ", e)

display_table_records(engine, employees)


SELECT employees.emp_no, employees.birth_date, employees.first_name, employees.last_name, employees.gender, employees.hire_date 
FROM employees
Table is empty
<class 'sqlalchemy.sql.dml.Insert'>
INSERT INTO employees (emp_no, birth_date, first_name, last_name, gender, hire_date) VALUES (:emp_no, :birth_date, :first_name, :last_name, :gender, :hire_date)
SELECT employees.emp_no, employees.birth_date, employees.first_name, employees.last_name, employees.gender, employees.hire_date 
FROM employees
['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
(1001, datetime.date(1985, 10, 12), 'John', 'Doe', 'M', datetime.date(2015, 9, 1))
SELECT employees.emp_no, employees.birth_date, employees.first_name, employees.last_name, employees.gender, employees.hire_date 
FROM employees
['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
(1001, datetime.date(1985, 10, 12), 'John', 'Doe', 'M', datetime.date(2015, 9, 1))
(1002, datetime.date(2001, 11, 23), 'Jan

In [4]:
# insert multiple records
value_list = [
    {'emp_no':1003, 'birth_date':'19760114', 'first_name':'Bruce', 'last_name':'Wayne', 'gender':'M', 'hire_date':'20000304'},
    {'emp_no':1004, 'birth_date':'19541024', 'first_name':'Alfred', 'last_name':'Pennyworth', 'gender':'M', 'hire_date':'20000304'},
    {'emp_no':1005, 'birth_date':'19871101', 'first_name':'Barbara', 'last_name':'Gordon', 'gender':'F', 'hire_date':'20120711'}
]

ins = db.insert(employees)

try:
    results = engine.execute(ins, value_list)
except BaseException as e:
    print("Error: ", e)


In [5]:
# select
display_table_records(engine, employees)

SELECT employees.emp_no, employees.birth_date, employees.first_name, employees.last_name, employees.gender, employees.hire_date 
FROM employees
['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
(1001, datetime.date(1985, 10, 12), 'John', 'Doe', 'M', datetime.date(2015, 9, 1))
(1002, datetime.date(2001, 11, 23), 'Jane', 'Doe', 'F', datetime.date(2020, 3, 1))
(1003, datetime.date(1976, 1, 14), 'Bruce', 'Wayne', 'M', datetime.date(2000, 3, 4))
(1004, datetime.date(1954, 10, 24), 'Alfred', 'Pennyworth', 'M', datetime.date(2000, 3, 4))
(1005, datetime.date(1987, 11, 1), 'Barbara', 'Gordon', 'F', datetime.date(2012, 7, 11))


In [6]:
# select with where clause
stmt = db.select(employees).where(employees.columns.gender == 'F')
results = engine.execute(stmt).fetchall()
for record in results:
    print(record)


(1002, datetime.date(2001, 11, 23), 'Jane', 'Doe', 'F', datetime.date(2020, 3, 1))
(1005, datetime.date(1987, 11, 1), 'Barbara', 'Gordon', 'F', datetime.date(2012, 7, 11))


In [7]:
stmt = db.select(employees).where(db.and_(employees.columns.gender == 'F',
                                      employees.columns.last_name == 'Doe'))
results = engine.execute(stmt).fetchall()
for record in results:
    print(record)


(1002, datetime.date(2001, 11, 23), 'Jane', 'Doe', 'F', datetime.date(2020, 3, 1))


In [8]:
# update
stmt = db.update(employees).values(birth_date = '20010504')
stmt = stmt.where(employees.columns.emp_no == 1002)

try:
    results = engine.execute(stmt)
except BaseException as e:
    print("Error: ", e)

display_table_records(engine, employees)


SELECT employees.emp_no, employees.birth_date, employees.first_name, employees.last_name, employees.gender, employees.hire_date 
FROM employees
['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
(1001, datetime.date(1985, 10, 12), 'John', 'Doe', 'M', datetime.date(2015, 9, 1))
(1002, datetime.date(2001, 5, 4), 'Jane', 'Doe', 'F', datetime.date(2020, 3, 1))
(1003, datetime.date(1976, 1, 14), 'Bruce', 'Wayne', 'M', datetime.date(2000, 3, 4))
(1004, datetime.date(1954, 10, 24), 'Alfred', 'Pennyworth', 'M', datetime.date(2000, 3, 4))
(1005, datetime.date(1987, 11, 1), 'Barbara', 'Gordon', 'F', datetime.date(2012, 7, 11))


In [34]:
# delete
stmt = db.delete(employees).where(
    db.and_(employees.columns.first_name == 'Alfred',
        employees.columns.gender == 'M'))
results = engine.execute(stmt)

print(results.rowcount)
display_table_records(engine, employees)


conn.close()

1
SELECT employees.emp_no, employees.birth_date, employees.first_name, employees.last_name, employees.gender, employees.hire_date 
FROM employees
['emp_no', 'birth_date', 'first_name', 'last_name', 'gender', 'hire_date']
(1001, datetime.date(1985, 10, 12), 'John', 'Doe', 'M', datetime.date(2015, 9, 1))
(1002, datetime.date(2001, 5, 4), 'Jane', 'Doe', 'F', datetime.date(2020, 3, 1))
(1003, datetime.date(1976, 1, 14), 'Bruce', 'Wayne', 'M', datetime.date(2000, 3, 4))
(1005, datetime.date(1987, 11, 1), 'Barbara', 'Gordon', 'F', datetime.date(2012, 7, 11))
